In [11]:
import scipy as sp
import numpy as np
import pandas as pd
import random
import csv

%load_ext autoreload
%autoreload 2
import Estimators as est
import importlib
importlib.reload(est)

np.set_printoptions(precision=2)

d = 3
dist_type = "exp"
est_type = "sw"
N=100000
total_repeat = 5
print("distribution=",dist_type, "estimitor=",est_type, 
      "total sample=",N, "total repeat num=",total_repeat)
x_max = 1
x_step_size = x_max/d
x_grid = x_step_size*np.arange(d)
print(x_grid)

eps_grid = 0.5*np.arange(1,6)
print(eps_grid)

if dist_type=='exp':
    x_q = np.exp(-3*x_grid)
    x_q = x_q/np.sum(x_q)
elif dist_type=='uni':
    x_q = np.ones(d)/d
else:
    print('warning: invaild distribution type!')
print(x_q)

filename = "data/%s_q_%d.csv" %(dist_type,d)
x_data = {'x': x_grid, 'q':x_q}
pd.DataFrame(x_data).to_csv(filename)
print(filename, 'saved')

idx_original = random.choices(np.arange(d),x_q,k=N)
sample_original = x_grid[idx_original]

for i in range(len(eps_grid)):
    eps= eps_grid[i]
    print('eps=%.2f'%(eps))

    if est_type == 'sw':
        a_grid, M = est.SquareWave(eps,x_grid)
    elif est_type == 'grr':
        a_grid, M = est.GenRandResp(eps, x_grid)
    else:
        print('warning: invalid estimator type!')

    filename = 'data/%s_M_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    filename = 'data/%s_a_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(a_grid).to_csv(filename)

    rand_num = np.zeros((N,d))
    for j in range(d):
        rand_num[:,j] = random.choices(a_grid,M[:,j],k=N)
    x_q_noisy = np.zeros((1,d))
    for k in range(total_repeat):
        idx_noise = random.choices(range(N),k=N)
        idx_pair = list(zip(idx_noise,idx_original))
        sample_perturbed = rand_num[tuple(zip(*idx_pair))]
        elements,counts = np.unique(sample_perturbed,return_counts=True)
        x_q_est = est.EM(M,counts,eps)
        x_q_noisy +=x_q_est
    x_q_noisy = x_q_noisy/np.sum(x_q_noisy)
    print("x_q_noisy=",x_q_noisy)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
distribution= exp estimitor= sw total sample= 100000 total repeat num= 5
[0.   0.33 0.67]
[0.5 1.  1.5 2.  2.5]
[0.67 0.24 0.09]
data/exp_q_3.csv saved
eps=0.50
x_q_noisy= [[0.65 0.28 0.08]]
eps=1.00
x_q_noisy= [[0.66 0.25 0.09]]
eps=1.50
x_q_noisy= [[0.66 0.24 0.09]]
eps=2.00
x_q_noisy= [[0.67 0.25 0.09]]
eps=2.50
x_q_noisy= [[0.66 0.25 0.09]]


In [19]:

def OptVariance(eps,x_grid,x_q):
    d = len(x_q)
    eEps = np.exp(eps)
    A_ub = np.zeros((d*(d-1)*d,d*d))
    print(A_ub.shape)

    counter = 0
    for i in range(d):
        for j in range(d):
            for k in range(d):
                A = np.zeros((d,d))
                if k!=j:
                    A[i,j] = 1
                    A[i,k]=-eEps
                    A = np.reshape(A,(1,d*d))
                    A_ub[counter,:] = A
                    counter+=1
    print(A_ub,shape)
    b_ub = np.zeros((1,d*d*(d-1)))
    print(b_ub,shape)
OptVariance(2,x_grid,x_q)

(18, 9)
[[ 1.   -7.39  0.    0.    0.    0.    0.    0.    0.  ]
 [ 1.    0.   -7.39  0.    0.    0.    0.    0.    0.  ]
 [-7.39  1.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.   -7.39  0.    0.    0.    0.    0.    0.  ]
 [-7.39  0.    1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.   -7.39  1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.   -7.39  0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.    0.   -7.39  0.    0.    0.  ]
 [ 0.    0.    0.   -7.39  1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    1.   -7.39  0.    0.    0.  ]
 [ 0.    0.    0.   -7.39  0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   -7.39  1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    1.   -7.39  0.  ]
 [ 0.    0.    0.    0.    0.    0.    1.    0.   -7.39]
 [ 0.    0.    0.    0.    0.    0.   -7.39  1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.   -7.39]
 [ 0.    0.    0.    0.    0.    0.   -7.39  0.    1.  ]
 [ 0.    0.    0.    0.